In [1]:
import tensorflow as tf
import numpy as np
from hourglass_tiny import HourglassModel
import configparser
import os
import pandas as pd
from keras.preprocessing import image
import scipy.io
from scipy.misc import imresize
import matplotlib.pyplot as plt
import math
from tqdm import tqdm

C:\Users\44163\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


In [2]:
# load data(we need the same inputs but (None,256,256,15) label)
data_path = os.path.join('Sitting','img')
label_path = os.path.join('Sitting','masks')
data_names = sorted(os.listdir(data_path))
label_names = sorted(os.listdir(label_path))
data_addr = []
for i in range(len(data_names)):
    data_addr.append(os.path.join('Sitting/img',data_names[i]))

label_addr = []
for i in range(len(label_names)):
    label_addr.append(os.path.join('Sitting/masks',label_names[i]))
The_Main_List = pd.DataFrame(data_names,columns = ['data_name'])
temp_frame1 = pd.DataFrame(label_names,columns=['label_name'])
temp_frame2 = pd.DataFrame(data_addr,columns=['data_addr'])
temp_frame3 = pd.DataFrame(label_addr,columns=['label_addr'])
The_Main_List = pd.concat([The_Main_List,temp_frame1],axis=1)
The_Main_List = pd.concat([The_Main_List,temp_frame2],axis=1)
The_Main_List = pd.concat([The_Main_List,temp_frame3],axis=1)
The_Main_List.head()

,data_name,label_name,data_addr,label_addr
0,image0.jpg,image0.mat,Sitting/img\image0.jpg,Sitting/masks\image0.mat
1,image1.jpg,image1.mat,Sitting/img\image1.jpg,Sitting/masks\image1.mat
2,image10.jpg,image10.mat,Sitting/img\image10.jpg,Sitting/masks\image10.mat
3,image100.jpg,image100.mat,Sitting/img\image100.jpg,Sitting/masks\image100.mat
4,image101.jpg,image101.mat,Sitting/img\image101.jpg,Sitting/masks\image101.mat


In [3]:
# load one image
def load_image(path):
    img = image.load_img(path,target_size=(256,256))
    img = image.img_to_array(img)
    return img

# load one label for upsampling(label size:(None,256,256,15))
def load_label_as_gtmap(path):
    label = scipy.io.loadmat(path)
    label = label['M']
    label_resize = imresize(label,[256,256],interp='nearest')
    label_resize = label_resize.astype('float32')
    label_floor = np.floor(label_resize/18)
    # transform to sigle map
    map = np.zeros([256,256,15])
    for i in range(15):
        for n in range(256):
            for m in range(256):
                if label_floor[n][m] == i:
                    map[n,m,i] = 1.0
    return map

In [4]:
# load all data
data = np.zeros([20,256,256,3])
for i,path in enumerate(The_Main_List.data_addr):
    data[i] = load_image(path)
    if i == 9:
        break
# load all labels
label = np.zeros([20,256,256,15])
for i,path in tqdm(enumerate(The_Main_List.label_addr)):
    label[i] = load_label_as_gtmap(path)
    if i == 9:
        break

0it [00:00, ?it/s]

C:\Users\44163\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':


1it [00:03,  3.26s/it]

2it [00:05,  2.88s/it]

3it [00:08,  2.69s/it]

...

5it [00:12,  2.52s/it]

6it [00:14,  2.49s/it]

7it [00:17,  2.46s/it]

8it [00:19,  2.43s/it]

9it [00:21,  2.42s/it]

In [5]:
# split train and test,first 80% train,last 20% test.
rnd = np.arange(len(data))
train_idx = rnd < (0.8*len(data))
valid_idx = rnd >= (0.8*len(data))
train_data = data[train_idx]
test_data = data[valid_idx]
train_label = label[train_idx]
test_label = label[valid_idx]

In [6]:
def next_batch(data,label,batch_size):
    batch_x = np.zeros([batch_size,256,256,3])
    batch_y = np.zeros([batch_size,256,256,15])
    idx = np.arange(len(data))
    np.random.shuffle(idx)
    for i in range(batch_size):
        batch_x[i] = data[idx[i]]
        batch_y[i] = label[idx[i]]
    return batch_x,batch_y

In [ ]:
# test the data
x,y = next_batch(train_data,train_label,1)
img = np.reshape(x,[256,256,3])
plt.figure()
plt.imshow(img/255)
plt.show()
label = np.reshape(y,[256,256,15])
label = np.argmax(label,axis=2)
plt.figure()
plt.imshow(label)
plt.show()

In [9]:
# build the model
def process_config(conf_file):
	"""
	"""
	params = {}
	config = configparser.ConfigParser()
	config.read(conf_file)
	for section in config.sections():
		if section == 'DataSetHG':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Network':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Train':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Validation':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Saver':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
	return params
print('--Parsing Config File')
params = process_config('config.cfg')

--Parsing Config File

In [10]:
def weight_variable(shape, stddev=0.02, name=None): #Create tensorflow matrix with  normal random distubotion mean 0 and standart deviation 0.02
    # print(shape)
    initial = tf.truncated_normal(shape, stddev=stddev)
    if name is None:
        return tf.Variable(initial)
    else:
        return tf.get_variable(name, initializer=initial)

def bias_variable(shape, name=None):
    initial = tf.constant(0.0, shape=shape)
    if name is None:
        return tf.Variable(initial)
    else:
        return tf.get_variable(name, initializer=initial)

def conv2d_basic(x, W, bias): #Simple conv and biase addition this function is waste of time replace in the code with the tensorflow command
    conv = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME") #Padding same mean the output is same size as input?
    return tf.nn.bias_add(conv, bias)

In [11]:
model = HourglassModel(nFeat=params['nfeats'], nStack=params['nstacks'], 
                       nModules=params['nmodules'], 
                       nLow=params['nlow'], outputDim=params['num_joints'], 
                       batch_size=params['batch_size'], attention = params['mcam'],
                       training=True, drop_rate= params['dropout_rate'], 
                       lear_rate=params['learning_rate'], 
                       decay=params['learning_rate_decay'], 
                       decay_step=params['decay_step'], 
                       name=params['name'], 
                       logdir_train=params['log_dir_train'], 
                       logdir_test=params['log_dir_test'], 
                       tiny= params['tiny'], w_loss=params['weighted_loss'] , 
                       joints= params['joint_list'],modif=False)
model.generate_model()

CREATE MODEL:

---Inputs : Done (0 sec.)

---Graph : Done (14 sec.)

---Loss : Done (0 sec.)

In [ ]:
# get the variables to restore
variables_to_restore = tf.contrib.slim.get_variables_to_restore()
# last map: (None,64,64,15)
output_stack = model.output
stacks = tf.unstack(output_stack,axis=1)
last_map = stacks[3]
# add layers for upsampling, from (64,64,15) to (256,256,15)
bi_upsampling = tf.image.resize_bilinear(last_map,[256,256],name='bilinear_upsampling')
W_last = weight_variable([3,3,15,15],name='W_last')
b_last = bias_variable([15],name='b_last')
aft_upsam_out = conv2d_basic(bi_upsampling,W_last,b_last)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aft_upsam_out, labels= model.hr_label), 
                      name= 'cross_entropy_loss')

In [ ]:
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
saver = tf.train.Saver(variables_to_restore)
path = "my_net"
full_path = os.path.join(path,'save_net.ckpt')
saver.restore(sess,full_path)

In [23]:
# training
for i in range(50):
    x,y = next_batch(train_data,train_label,1)
    sess.run(train_step,feed_dict={model.img:x, model.hr_label:y})
    print(sess.run(loss,feed_dict={model.img:x, model.hr_label:y}))

0.35137513

0.24777457

0.18878126

0.16092032

0.13976334

KeyboardInterrupt: 

In [ ]:
# save the net
# save the network
path = "my_upsampling_net"
is_exist = os.path.exists(path)
if not is_exist:
    os.makedirs(path)
full_path = os.path.join(path,'save_net.ckpt')
save_path = saver.save(sess,full_path)